https://dota2.ru/esport/matches/

In [23]:
import requests
import time
import pandas as pd
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
import gc
import os
from pathlib import Path

## Парсинг адресов завершенных матчей

In [2]:
# Функция для составления списка с адрессами на завершенные матчи одной страницы
def matches_on_one_page(url_page):
    matches_one_page = []
    response = requests.get(url_page, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(response.text, 'html.parser')
    # Выделим все матчи на странице
    all_matches = soup.find_all(
        'a', class_='cybersport-matches__matches-match-wrap')

    # на первой странице под один класс записаны текущие, будущие и прошедшие матчи (выделим только прошедшие)
    if '1/' in url_page:
        for i in range(len(all_matches)):
            if all_matches[i].find('div', "cybersport-matches__matches-time").get_text(strip=True) != '':
                continue
            else:
                matches_one_page.append(
                    'https://dota2.ru'+all_matches[i]['href'])
    else:
        for i in range(len(all_matches)):
            matches_one_page.append('https://dota2.ru'+all_matches[i]['href'])
    return matches_one_page

In [3]:
# Функция для парсинга адрессов завершенных матчей страницы на выбор (если interval=True(default), то будут парситься страницы
# с 1 до pages+1, если interval=False, то будет парситься только одна страница, указанная в pages)
# start_page - первая траница для инвервала
def all_matches(end_page, interval=True, start_page=1):
    all_matches = []
    if interval == True:
        for i in tqdm(range(start_page, end_page+1)):
            url_page = 'https://dota2.ru/esport/matches/?page='+str(i)+'/'
            matches_on_one_page(url_page)
            all_matches.extend(matches_on_one_page(url_page))
    elif interval == False:
        url_page = 'https://dota2.ru/esport/matches/?page='+str(pages)+'/'
        matches_on_one_page(url_page)
        all_matches.extend(matches_on_one_page(url_page))
    return all_matches

In [4]:
# Всего 553 страницы на 24.08.2021
matches_ids = all_matches(2, interval=True, start_page=1)
gc.collect()

45303

## Парсинг информации для одного матча по URL


*Парсинг для данных категорий будет отличаться:*
- bo3: сыграно 2 или 3 карты
- bo5: сыграно 3, 4 или 5 карт
- есть полная информация по каждой карте
- есть данные только по одной карте
- неполная информация

In [5]:
def parse(match_id):
    df = {'match': [],
          'pinnacle': '-',
          'parimatch': '-',
          'ggbet': '-',
          'date': [],
          'tournament': [],
          'team left': [],
          'players_left': [],
          'team right': [],
          'players_right': [],
          'ban_left': [],
          'ban_right': [],
          'pick_left_team': [],
          'pick_right_team': [],
          'result_left_team': [],
          'result_right_team': [],
          'score': []}
    response = requests.get(match_id, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(response.text, 'html.parser')
    if response.status_code == 200:
        # bo3
        if soup.find('div', 'cybersport-matches-match__preview-teams-score').get_text(strip=True) == 'Best of 3':
            if ((soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').get_text(strip=True) != 'Игры не было') &
                    (soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').get_text(strip=True) != 'Игры не было') &
                (soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find(
                    'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0') &
                (soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find(
                    'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0')):
                # match
                df['match'] = soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker-team')[0].get_text(
                    strip=True) + ' vs ' + soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker-team')[1].get_text(strip=True)
                # odds
                if 'pinnacle' in soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[0].find('img')['src']:
                    df['pinnacle'] = soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[0].find(
                        'div', class_="cybersport-matches-match__coeff-bookmaker-coeff").get_text(strip=True)
                elif 'parimatch' in soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[0].find('img')['src']:
                    df['parimatch'] = soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[0].find(
                        'div', class_="cybersport-matches-match__coeff-bookmaker-coeff").get_text(strip=True)
                elif 'ggbet' in soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[0].find('img')['src']:
                    df['ggbet'] = soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[0].find(
                        'div', class_="cybersport-matches-match__coeff-bookmaker-coeff").get_text(strip=True)
                if len(soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')) > 1:
                    if 'pinnacle' in soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[1].find('img')['src']:
                        df['pinnacle'] = soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[1].find(
                            'div', class_="cybersport-matches-match__coeff-bookmaker-coeff").get_text(strip=True)
                    elif 'parimatch' in soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[1].find('img')['src']:
                        df['parimatch'] = soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[1].find(
                            'div', class_="cybersport-matches-match__coeff-bookmaker-coeff").get_text(strip=True)
                    elif 'ggbet' in soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[1].find('img')['src']:
                        df['ggbet'] = soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[1].find(
                            'div', class_="cybersport-matches-match__coeff-bookmaker-coeff").get_text(strip=True)
                # date
                df['date'] = soup.find_all(
                    'div', class_='cybersport-matches-match__preview-descr-text')[-1].get_text(strip=True).replace('Завершен / ', '') + ' 2021'
                # tournament
                df['tournament'] = soup.find(
                    'a', class_='cybersport-matches-match__preview-descr-name').get_text(strip=True)
                # team left
                # 1 map
                df['team left'].append([x.get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find_all('p', class_='cybersport-matches__matches-name')][0].replace('Победа', ''))
                # 2 map
                df['team left'].append([x.get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find_all('p', class_='cybersport-matches__matches-name')][0].replace('Победа', ''))
                # 3 map
                if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').get_text(strip=True) != 'Игры не было':
                    if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').find(
                            'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0':
                        df['team left'].append([x.get_text(strip=True) for x in soup.find(
                            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').find_all('p', class_='cybersport-matches__matches-name')][0].replace('Победа', ''))
                # players left
                # 1 map
                df['players_left'].append([x.find('a').get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find_all('div', class_='global-content__tabel--left')[0:5]])
                # 2 map
                df['players_left'].append([x.find('a').get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find_all('div', class_='global-content__tabel--left')[0:5]])
                # 3 map
                if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').get_text(strip=True) != 'Игры не было':
                    if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').find(
                            'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0':
                        df['players_left'].append([x.find('a').get_text(strip=True) for x in soup.find(
                            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').find_all('div', class_='global-content__tabel--left')[0:5]])
                # team right
                # 1 map
                df['team right'].append([x.get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find_all('p', class_='cybersport-matches__matches-name')][1].replace('Победа', ''))
                # 2 map
                df['team right'].append([x.get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find_all('p', class_='cybersport-matches__matches-name')][1].replace('Победа', ''))
                # 3 map
                if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').get_text(strip=True) != 'Игры не было':
                    if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').find(
                            'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0':
                        df['team right'].append([x.get_text(strip=True) for x in soup.find(
                            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').find_all('p', class_='cybersport-matches__matches-name')][1].replace('Победа', ''))
                # players right
                # 1 map
                df['players_right'].append([x.find('a').get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find_all('div', class_='global-content__tabel--left')[5:10]])
                # 2 map
                df['players_right'].append([x.find('a').get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find_all('div', class_='global-content__tabel--left')[5:10]])
                # 3 map
                if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').get_text(strip=True) != 'Игры не было':
                    if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').find(
                            'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0':
                        df['players_right'].append([x.find('a').get_text(strip=True) for x in soup.find(
                            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').find_all('div', class_='global-content__tabel--left')[5:10]])
                # ban left
                # 1 map
                df['ban_left'].append([y['title'].replace('Бан ', '') for y in ([x for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find_all('div', class_='global-content__tabel--right')][0].find_all('a'))])
                # 2 map
                df['ban_left'].append([y['title'].replace('Бан ', '') for y in ([x for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find_all('div', class_='global-content__tabel--right')][0].find_all('a'))])
                # 3 map
                if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').get_text(strip=True) != 'Игры не было':
                    if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').find(
                            'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0':
                        df['ban_left'].append([y['title'].replace('Бан ', '') for y in ([x for x in soup.find(
                            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').find_all('div', class_='global-content__tabel--right')][0].find_all('a'))])
                # ban right
                # 1 map
                df['ban_right'].append([y['title'].replace('Бан ', '') for y in ([x for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find_all('div', class_='global-content__tabel--right')][1].find_all('a'))])
                # 2 map
                df['ban_right'].append([y['title'].replace('Бан ', '') for y in ([x for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find_all('div', class_='global-content__tabel--right')][1].find_all('a'))])
                # 3 map
                if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').get_text(strip=True) != 'Игры не было':
                    if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').find(
                            'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0':
                        df['ban_right'].append([y['title'].replace('Бан ', '') for y in ([x for x in soup.find(
                            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').find_all('div', class_='global-content__tabel--right')][1].find_all('a'))])
                # pick left team
                # 1 map
                df['pick_left_team'].append([x.get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find_all('span', class_='table__hero-name')][0:5])
                # 2 map
                df['pick_left_team'].append([x.get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find_all('span', class_='table__hero-name')][0:5])
                # 3 map
                if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').get_text(strip=True) != 'Игры не было':
                    if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').find(
                            'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0':
                        df['pick_left_team'].append([x.get_text(strip=True) for x in soup.find(
                            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').find_all('span', class_='table__hero-name')][0:5])
                # pick right team
                # 1 map
                df['pick_right_team'].append([x.get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find_all('span', class_='table__hero-name')][5:10])
                # 2 map
                df['pick_right_team'].append([x.get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find_all('span', class_='table__hero-name')][5:10])
                # 3 map
                if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').get_text(strip=True) != 'Игры не было':
                    if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').find(
                            'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0':
                        df['pick_right_team'].append([x.get_text(strip=True) for x in soup.find(
                            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').find_all('span', class_='table__hero-name')][5:10])
                # result left team
                # 1 map
                df['result_left_team'].append([x.find('span') for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find_all('div', class_='cybersport-matches__matches-block-wrap')][0])
                # 2 map
                df['result_left_team'].append([x.find('span') for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find_all('div', class_='cybersport-matches__matches-block-wrap')][0])
                # 3 map
                if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').get_text(strip=True) != 'Игры не было':
                    if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').find(
                            'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0':
                        df['result_left_team'].append([x.find('span') for x in soup.find(
                            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').find_all('div', class_='cybersport-matches__matches-block-wrap')][0])
                # result right team
                # 1 map
                df['result_right_team'].append([x.find('span') for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find_all('div', class_='cybersport-matches__matches-block-wrap')][1])
                # 2 map
                df['result_right_team'].append([x.find('span') for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find_all('div', class_='cybersport-matches__matches-block-wrap')][1])
                # 3 map
                if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').get_text(strip=True) != 'Игры не было':
                    if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').find(
                            'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0':
                        df['result_right_team'].append([x.find('span') for x in soup.find(
                            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').find_all('div', class_='cybersport-matches__matches-block-wrap')][1])
                # score
                # 1 map
                df['score'].append(soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find(
                    'div', 'cybersport-matches__matches-time').get_text(strip=True))
                # 2 map
                df['score'].append(soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find(
                    'div', 'cybersport-matches__matches-time').get_text(strip=True))
                # 3 map
                if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').get_text(strip=True) != 'Игры не было':
                    if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').find(
                            'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0':
                        df['score'].append(soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').find(
                            'div', 'cybersport-matches__matches-time').get_text(strip=True))
                return df
        # bo2
        elif soup.find('div', 'cybersport-matches-match__preview-teams-score').get_text(strip=True) == 'Best of 2':
            if ((soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').get_text(strip=True) != 'Игры не было') &
                    (soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').get_text(strip=True) != 'Игры не было') &
                (soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find(
                    'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0') &
                (soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find(
                    'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0')):
                # match
                df['match'] = soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker-team')[0].get_text(
                    strip=True) + ' vs ' + soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker-team')[1].get_text(strip=True)
                # odds
                if 'pinnacle' in soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[0].find('img')['src']:
                    df['pinnacle'] = soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[0].find(
                        'div', class_="cybersport-matches-match__coeff-bookmaker-coeff").get_text(strip=True)
                elif 'parimatch' in soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[0].find('img')['src']:
                    df['parimatch'] = soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[0].find(
                        'div', class_="cybersport-matches-match__coeff-bookmaker-coeff").get_text(strip=True)
                elif 'ggbet' in soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[0].find('img')['src']:
                    df['ggbet'] = soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[0].find(
                        'div', class_="cybersport-matches-match__coeff-bookmaker-coeff").get_text(strip=True)
                if len(soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')) > 1:
                    if 'pinnacle' in soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[1].find('img')['src']:
                        df['pinnacle'] = soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[1].find(
                            'div', class_="cybersport-matches-match__coeff-bookmaker-coeff").get_text(strip=True)
                    elif 'parimatch' in soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[1].find('img')['src']:
                        df['parimatch'] = soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[1].find(
                            'div', class_="cybersport-matches-match__coeff-bookmaker-coeff").get_text(strip=True)
                    elif 'ggbet' in soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[1].find('img')['src']:
                        df['ggbet'] = soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[1].find(
                            'div', class_="cybersport-matches-match__coeff-bookmaker-coeff").get_text(strip=True)
                # date
                df['date'] = soup.find_all(
                    'div', class_='cybersport-matches-match__preview-descr-text')[-1].get_text(strip=True).replace('Завершен / ', '') + ' 2021'
                # tournament
                df['tournament'] = soup.find(
                    'a', class_='cybersport-matches-match__preview-descr-name').get_text(strip=True)
                # team left
                # 1 map
                df['team left'].append([x.get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find_all('p', class_='cybersport-matches__matches-name')][0].replace('Победа', ''))
                # 2 map
                df['team left'].append([x.get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find_all('p', class_='cybersport-matches__matches-name')][0].replace('Победа', ''))
                # players left
                # 1 map
                df['players_left'].append([x.find('a').get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find_all('div', class_='global-content__tabel--left')[0:5]])
                # 2 map
                df['players_left'].append([x.find('a').get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find_all('div', class_='global-content__tabel--left')[0:5]])
                # team right
                # 1 map
                df['team right'] = [[x.get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find_all('p', class_='cybersport-matches__matches-name')][1].replace('Победа', '')]
                # 2 map
                df['team right'].append([x.get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find_all('p', class_='cybersport-matches__matches-name')][1].replace('Победа', ''))
                # players right
                # 1 map
                df['players_right'] = [[x.find('a').get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find_all('div', class_='global-content__tabel--left')[5:10]]]
                # 2 map
                df['players_right'].append([x.find('a').get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find_all('div', class_='global-content__tabel--left')[5:10]])
                # ban left
                # 1 map
                df['ban_left'].append([y['title'].replace('Бан ', '') for y in ([x for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find_all('div', class_='global-content__tabel--right')][0].find_all('a'))])
                # 2 map
                df['ban_left'].append([y['title'].replace('Бан ', '') for y in ([x for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find_all('div', class_='global-content__tabel--right')][0].find_all('a'))])
                # ban right
                # 1 map
                df['ban_right'].append([y['title'].replace('Бан ', '') for y in ([x for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find_all('div', class_='global-content__tabel--right')][1].find_all('a'))])
                # 2 map
                df['ban_right'].append([y['title'].replace('Бан ', '') for y in ([x for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find_all('div', class_='global-content__tabel--right')][1].find_all('a'))])
                # pick left team
                # 1 map
                df['pick_left_team'].append([x.get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find_all('span', class_='table__hero-name')][0:5])
                # 2 map
                df['pick_left_team'].append([x.get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find_all('span', class_='table__hero-name')][0:5])
                # pick right team
                # 1 map
                df['pick_right_team'].append([x.get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find_all('span', class_='table__hero-name')][5:10])
                # 2 map
                df['pick_right_team'].append([x.get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find_all('span', class_='table__hero-name')][5:10])
                # result left team
                # 1 map
                df['result_left_team'].append([x.find('span') for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find_all('div', class_='cybersport-matches__matches-block-wrap')][0])
                # 2 map
                df['result_left_team'].append([x.find('span') for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find_all('div', class_='cybersport-matches__matches-block-wrap')][0])
                # result right team
                # 1 map
                df['result_right_team'].append([x.find('span') for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find_all('div', class_='cybersport-matches__matches-block-wrap')][1])
                # 2 map
                df['result_right_team'].append([x.find('span') for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find_all('div', class_='cybersport-matches__matches-block-wrap')][1])
                # score
                # 1 map
                df['score'].append(soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find(
                    'div', 'cybersport-matches__matches-time').get_text(strip=True))
                # 2 map
                df['score'].append(soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find(
                    'div', 'cybersport-matches__matches-time').get_text(strip=True))
                return df
        # bo5
        elif soup.find('div', 'cybersport-matches-match__preview-teams-score').get_text(strip=True) == 'Best of 5':
            if ((soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').get_text(strip=True) != 'Игры не было') &
                    (soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').get_text(strip=True) != 'Игры не было') &
                    (soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').get_text(strip=True) != 'Игры не было') &
                (soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find(
                    'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0') &
                (soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find(
                    'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0') &
                (soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').find(
                    'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0')):
                # match
                df['match'] = soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker-team')[0].get_text(
                    strip=True) + ' vs ' + soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker-team')[1].get_text(strip=True)
                # odds
                if 'pinnacle' in soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[0].find('img')['src']:
                    df['pinnacle'] = soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[0].find(
                        'div', class_="cybersport-matches-match__coeff-bookmaker-coeff").get_text(strip=True)
                elif 'parimatch' in soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[0].find('img')['src']:
                    df['parimatch'] = soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[0].find(
                        'div', class_="cybersport-matches-match__coeff-bookmaker-coeff").get_text(strip=True)
                elif 'ggbet' in soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[0].find('img')['src']:
                    df['ggbet'] = soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[0].find(
                        'div', class_="cybersport-matches-match__coeff-bookmaker-coeff").get_text(strip=True)
                if len(soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')) > 1:
                    if 'pinnacle' in soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[1].find('img')['src']:
                        df['pinnacle'] = soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[1].find(
                            'div', class_="cybersport-matches-match__coeff-bookmaker-coeff").get_text(strip=True)
                    elif 'parimatch' in soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[1].find('img')['src']:
                        df['parimatch'] = soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[1].find(
                            'div', class_="cybersport-matches-match__coeff-bookmaker-coeff").get_text(strip=True)
                    elif 'ggbet' in soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[1].find('img')['src']:
                        df['ggbet'] = soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[1].find(
                            'div', class_="cybersport-matches-match__coeff-bookmaker-coeff").get_text(strip=True)
                # date
                df['date'] = soup.find_all(
                    'div', class_='cybersport-matches-match__preview-descr-text')[-1].get_text(strip=True).replace('Завершен / ', '') + ' 2021'
                # tournament
                df['tournament'] = soup.find(
                    'a', class_='cybersport-matches-match__preview-descr-name').get_text(strip=True)
                # team left
                # 1 map
                df['team left'].append([x.get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find_all('p', class_='cybersport-matches__matches-name')][0].replace('Победа', ''))
                # 2 map
                df['team left'].append([x.get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find_all('p', class_='cybersport-matches__matches-name')][0].replace('Победа', ''))
                # 3 map
                df['team left'].append([x.get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').find_all('p', class_='cybersport-matches__matches-name')][0].replace('Победа', ''))
                # 4 map
                if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--4').get_text(strip=True) != 'Игры не было':
                    if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--4').find(
                            'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0':
                        df['team left'].append([x.get_text(strip=True) for x in soup.find(
                            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--4').find_all('p', class_='cybersport-matches__matches-name')][0].replace('Победа', ''))
                # 5 map
                if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--5').get_text(strip=True) != 'Игры не было':
                    if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--5').find(
                            'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0':
                        df['team left'].append([x.get_text(strip=True) for x in soup.find(
                            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--5').find_all('p', class_='cybersport-matches__matches-name')][0].replace('Победа', ''))
                # players left
                # 1 map
                df['players_left'].append([x.find('a').get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find_all('div', class_='global-content__tabel--left')[0:5]])
                # 2 map
                df['players_left'].append([x.find('a').get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find_all('div', class_='global-content__tabel--left')[0:5]])
                # 3 map
                df['players_left'].append([x.find('a').get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').find_all('div', class_='global-content__tabel--left')[0:5]])
                # 4 map
                if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--4').get_text(strip=True) != 'Игры не было':
                    if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--4').find(
                            'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0':
                        df['players_left'].append([x.find('a').get_text(strip=True) for x in soup.find(
                            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--4').find_all('div', class_='global-content__tabel--left')[0:5]])
                # 5 map
                if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--5').get_text(strip=True) != 'Игры не было':
                    if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--5').find(
                            'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0':
                        df['players_left'].append([x.find('a').get_text(strip=True) for x in soup.find(
                            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--4').find_all('div', class_='global-content__tabel--left')[0:5]])
                # team right
                # 1 map
                df['team right'].append([x.get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find_all('p', class_='cybersport-matches__matches-name')][1].replace('Победа', ''))
                # 2 map
                df['team right'].append([x.get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find_all('p', class_='cybersport-matches__matches-name')][1].replace('Победа', ''))
                # 3 map
                df['team right'].append([x.get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').find_all('p', class_='cybersport-matches__matches-name')][1].replace('Победа', ''))
                # 4 map
                if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--4').get_text(strip=True) != 'Игры не было':
                    if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--4').find(
                            'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0':
                        df['team right'].append([x.get_text(strip=True) for x in soup.find(
                            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--4').find_all('p', class_='cybersport-matches__matches-name')][1].replace('Победа', ''))
                # 5 map
                if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--5').get_text(strip=True) != 'Игры не было':
                    if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--5').find(
                            'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0':
                        df['team right'].append([x.get_text(strip=True) for x in soup.find(
                            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--5').find_all('p', class_='cybersport-matches__matches-name')][1].replace('Победа', ''))
                # players right
                # 1 map
                df['players_right'].append([x.find('a').get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find_all('div', class_='global-content__tabel--left')[5:10]])
                # 2 map
                df['players_right'].append([x.find('a').get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find_all('div', class_='global-content__tabel--left')[5:10]])
                # 3 map
                df['players_right'].append([x.find('a').get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').find_all('div', class_='global-content__tabel--left')[5:10]])
                # 4 map
                if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--4').get_text(strip=True) != 'Игры не было':
                    if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--4').find(
                            'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0':
                        df['players_right'].append([x.find('a').get_text(strip=True) for x in soup.find(
                            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--4').find_all('div', class_='global-content__tabel--left')[5:10]])
                # 5 map
                if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--5').get_text(strip=True) != 'Игры не было':
                    if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--5').find(
                            'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0':
                        df['players_right'].append([x.find('a').get_text(strip=True) for x in soup.find(
                            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--5').find_all('div', class_='global-content__tabel--left')[5:10]])
                # ban left
                # 1 map
                df['ban_left'].append([y['title'].replace('Бан ', '') for y in ([x for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find_all('div', class_='global-content__tabel--right')][0].find_all('a'))])
                # 2 map
                df['ban_left'].append([y['title'].replace('Бан ', '') for y in ([x for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find_all('div', class_='global-content__tabel--right')][0].find_all('a'))])
                # 3 map
                df['ban_left'].append([y['title'].replace('Бан ', '') for y in ([x for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').find_all('div', class_='global-content__tabel--right')][0].find_all('a'))])
                # 4 map
                if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--4').get_text(strip=True) != 'Игры не было':
                    if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--4').find(
                            'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0':
                        df['ban_left'].append([y['title'].replace('Бан ', '') for y in ([x for x in soup.find(
                            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--4').find_all('div', class_='global-content__tabel--right')][0].find_all('a'))])
                # 5 map
                if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--5').get_text(strip=True) != 'Игры не было':
                    if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--5').find(
                            'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0':
                        df['ban_left'].append([y['title'].replace('Бан ', '') for y in ([x for x in soup.find(
                            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--5').find_all('div', class_='global-content__tabel--right')][0].find_all('a'))])
                # ban right
                # 1 map
                df['ban_right'].append([y['title'].replace('Бан ', '') for y in ([x for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find_all('div', class_='global-content__tabel--right')][1].find_all('a'))])
                # 2 map
                df['ban_right'].append([y['title'].replace('Бан ', '') for y in ([x for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find_all('div', class_='global-content__tabel--right')][1].find_all('a'))])
                # 3 map
                df['ban_right'].append([y['title'].replace('Бан ', '') for y in ([x for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').find_all('div', class_='global-content__tabel--right')][1].find_all('a'))])
                # 4 map
                if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--4').get_text(strip=True) != 'Игры не было':
                    if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--4').find(
                            'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0':
                        df['ban_right'].append([y['title'].replace('Бан ', '') for y in ([x for x in soup.find(
                            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--4').find_all('div', class_='global-content__tabel--right')][1].find_all('a'))])
                # 5 map
                if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--5').get_text(strip=True) != 'Игры не было':
                    if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--5').find(
                            'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0':
                        df['ban_right'].append([y['title'].replace('Бан ', '') for y in ([x for x in soup.find(
                            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--5').find_all('div', class_='global-content__tabel--right')][1].find_all('a'))])
                # pick left team
                # 1 map
                df['pick_left_team'].append([x.get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find_all('span', class_='table__hero-name')][0:5])
                # 2 map
                df['pick_left_team'].append([x.get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find_all('span', class_='table__hero-name')][0:5])
                # 3 map
                df['pick_left_team'].append([x.get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').find_all('span', class_='table__hero-name')][0:5])
                # 4 map
                if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--4').get_text(strip=True) != 'Игры не было':
                    if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--4').find(
                            'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0':
                        df['pick_left_team'].append([x.get_text(strip=True) for x in soup.find(
                            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--4').find_all('span', class_='table__hero-name')][0:5])
                # 5 map
                if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--5').get_text(strip=True) != 'Игры не было':
                    if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--5').find(
                            'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0':
                        df['pick_left_team'].append([x.get_text(strip=True) for x in soup.find(
                            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--5').find_all('span', class_='table__hero-name')][0:5])
                # pick right team
                # 1 map
                df['pick_right_team'].append([x.get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find_all('span', class_='table__hero-name')][5:10])
                # 2 map
                df['pick_right_team'].append([x.get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find_all('span', class_='table__hero-name')][5:10])
                # 3 map
                df['pick_right_team'].append([x.get_text(strip=True) for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').find_all('span', class_='table__hero-name')][5:10])
                # 4 map
                if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--4').get_text(strip=True) != 'Игры не было':
                    if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--4').find(
                            'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0':
                        df['pick_right_team'].append([x.get_text(strip=True) for x in soup.find(
                            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--4').find_all('span', class_='table__hero-name')][5:10])
                # 5 map
                if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--5').get_text(strip=True) != 'Игры не было':
                    if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--5').find(
                            'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0':
                        df['pick_right_team'].append([x.get_text(strip=True) for x in soup.find(
                            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--5').find_all('span', class_='table__hero-name')][5:10])
                # result left team
                # 1 map
                df['result_left_team'].append([x.find('span') for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find_all('div', class_='cybersport-matches__matches-block-wrap')][0])
                # 2 map
                df['result_left_team'].append([x.find('span') for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find_all('div', class_='cybersport-matches__matches-block-wrap')][0])
                # 3 map
                df['result_left_team'].append([x.find('span') for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').find_all('div', class_='cybersport-matches__matches-block-wrap')][0])
                # 4 map
                if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--4').get_text(strip=True) != 'Игры не было':
                    if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--4').find(
                            'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0':
                        df['result_left_team'].append([x.find('span') for x in soup.find(
                            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--4').find_all('div', class_='cybersport-matches__matches-block-wrap')][0])
                # 5 map
                if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--5').get_text(strip=True) != 'Игры не было':
                    if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--5').find(
                            'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0':
                        df['result_left_team'].append([x.find('span') for x in soup.find(
                            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--5').find_all('div', class_='cybersport-matches__matches-block-wrap')][0])
                # result right team
                # 1 map
                df['result_right_team'].append([x.find('span') for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find_all('div', class_='cybersport-matches__matches-block-wrap')][1])
                # 2 map
                df['result_right_team'].append([x.find('span') for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find_all('div', class_='cybersport-matches__matches-block-wrap')][1])
                # 3 map
                df['result_right_team'].append([x.find('span') for x in soup.find(
                    'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').find_all('div', class_='cybersport-matches__matches-block-wrap')][1])
                # 4 map
                if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--4').get_text(strip=True) != 'Игры не было':
                    if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--4').find(
                            'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0':
                        df['result_right_team'].append([x.find('span') for x in soup.find(
                            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--4').find_all('div', class_='cybersport-matches__matches-block-wrap')][1])
                # 5 map
                if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--5').get_text(strip=True) != 'Игры не было':
                    if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--5').find(
                            'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0':
                        df['result_right_team'].append([x.find('span') for x in soup.find(
                            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--5').find_all('div', class_='cybersport-matches__matches-block-wrap')][1])
                # score
                # 1 map
                df['score'].append(soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--1').find(
                    'div', 'cybersport-matches__matches-time').get_text(strip=True))
                # 2 map
                df['score'].append(soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--2').find(
                    'div', 'cybersport-matches__matches-time').get_text(strip=True))
                # 3 map
                df['score'].append(soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--3').find(
                    'div', 'cybersport-matches__matches-time').get_text(strip=True))
                # 4 map
                if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--4').get_text(strip=True) != 'Игры не было':
                    if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--4').find(
                            'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0':
                        df['score'].append(soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--4').find(
                            'div', 'cybersport-matches__matches-time').get_text(strip=True))
                # 5 map
                if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--5').get_text(strip=True) != 'Игры не было':
                    if soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--5').find(
                            'span', class_='score__cup js-score-cup js-esport-map-stats-score-cup').get_text(strip=True) != '0 : 0':
                        df['score'].append(soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--5').find(
                            'div', 'cybersport-matches__matches-time').get_text(strip=True))
                return df
    else:
        print('Ошибка с сайта! Ждем 100 секунд!')
        time.sleep(100)
        

def parse_live(match_id, num_map):
    df = {'match': [],
          'pinnacle': '-',
          'parimatch': '-',
          'ggbet': '-',
          'date': [],
          'tournament': [],
          'team left': [],
          'players_left': [],
          'team right': [],
          'players_right': [],
          'ban_left': [],
          'ban_right': [],
          'pick_left_team': [],
          'pick_right_team': [],
          'result_left_team': [],
          'result_right_team': [],
          'score': []}
    response = requests.get(match_id, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(response.text, 'html.parser')
    if response.status_code == 200:
        # bo3
        # match
        df['match'] = soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker-team')[0].get_text(
                            strip=True) + ' vs ' + soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker-team')[1].get_text(strip=True)
        # odds
        if 'pinnacle' in soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[0].find('img')['src']:
            df['pinnacle'] = soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[0].find(
                'div', class_="cybersport-matches-match__coeff-bookmaker-coeff").get_text(strip=True)
        elif 'parimatch' in soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[0].find('img')['src']:
            df['parimatch'] = soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[0].find(
                'div', class_="cybersport-matches-match__coeff-bookmaker-coeff").get_text(strip=True)
        elif 'ggbet' in soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[0].find('img')['src']:
            df['ggbet'] = soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[0].find(
                'div', class_="cybersport-matches-match__coeff-bookmaker-coeff").get_text(strip=True)
        if len(soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')) > 1:
            if 'pinnacle' in soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[1].find('img')['src']:
                df['pinnacle'] = soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[1].find(
                    'div', class_="cybersport-matches-match__coeff-bookmaker-coeff").get_text(strip=True)
            elif 'parimatch' in soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[1].find('img')['src']:
                df['parimatch'] = soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[1].find(
                    'div', class_="cybersport-matches-match__coeff-bookmaker-coeff").get_text(strip=True)
            elif 'ggbet' in soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[1].find('img')['src']:
                df['ggbet'] = soup.find_all('div', class_='cybersport-matches-match__coeff-bookmaker')[1].find(
                    'div', class_="cybersport-matches-match__coeff-bookmaker-coeff").get_text(strip=True)
        # date
        df['date'] = soup.find_all(
            'div', class_='cybersport-matches-match__preview-descr-text')[-1].get_text(strip=True).replace('Прямая трансляция', '').replace('Завершен / ', '') + ' 2021'
        # tournament
        df['tournament'] = soup.find(
                'a', class_='cybersport-matches-match__preview-descr-name').get_text(strip=True)
        # team left
        df['team left'].append([x.get_text(strip=True) for x in soup.find(
            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--'+str(num_map)).find_all('p', class_='cybersport-matches__matches-name')][0].replace('Победа', ''))
        # players left
        df['players_left'].append([x.find('a').get_text(strip=True) for x in soup.find(
            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--'+str(num_map)).find_all('div', class_='global-content__tabel--left')[0:5]])
        # team right
        df['team right'] = [[x.get_text(strip=True) for x in soup.find(
            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--'+str(num_map)).find_all('p', class_='cybersport-matches__matches-name')][1].replace('Победа', '')]
        # players right
        df['players_right'].append([x.find('a').get_text(strip=True) for x in soup.find(
            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--'+str(num_map)).find_all('div', class_='global-content__tabel--left')[5:10]])
        # ban left
        df['ban_left'].append([y['title'].replace('Бан ', '') for y in ([x for x in soup.find(
            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--'+str(num_map)).find_all('div', class_='global-content__tabel--right')][0].find_all('a'))])
         # ban right
        df['ban_right'].append([y['title'].replace('Бан ', '') for y in ([x for x in soup.find(
            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--'+str(num_map)).find_all('div', class_='global-content__tabel--right')][1].find_all('a'))])
        # pick left team
        df['pick_left_team'].append([x.get_text(strip=True) for x in soup.find(
            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--'+str(num_map)).find_all('span', class_='table__hero-name')][0:5])
        # pick right team
        df['pick_right_team'].append([x.get_text(strip=True) for x in soup.find(
            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--'+str(num_map)).find_all('span', class_='table__hero-name')][5:10])
        # result left team
        df['result_left_team'].append([x.find('span') for x in soup.find(
            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--'+str(num_map)).find_all('div', class_='cybersport-matches__matches-block-wrap')][0])
        # result right team
        df['result_right_team'].append([x.find('span') for x in soup.find(
            'div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--'+str(num_map)).find_all('div', class_='cybersport-matches__matches-block-wrap')][1])
        # score
        df['score'].append(soup.find('div', 'cybersport-matches-match__record-game-block cybersport-matches-match__record-game-block--'+str(num_map)).find(
            'div', 'cybersport-matches__matches-time').get_text(strip=True))
                   
    return df

## Парсинг

Функция для парсинга матчей и вывода всего дата фрейма
**parse_all(list_of_urls=None, export=False)**
- data : DataFrame
- export : bool, default False {dumps the final DataFrame}
- pages : string {interval of parsing pages}

In [6]:
def parse_all(list_of_urls, export=True, pages=''):
    data = pd.DataFrame()
    for match in tqdm(list_of_urls):
        try:
            df = parse(match)
            data = data.append(pd.DataFrame(df), ignore_index=True)
        except:
            pass
    if export == True:
        data.to_csv(r'C:\Users\Максим\Dota 2_PY_Project\parse\Dota_2_matches_'+pages+'.csv', index=False)
    return data

In [7]:
parse_all(matches_ids, pages='11_09_2021')
gc.collect()

725114

In [51]:
predict = parse('https://dota2.ru/esport/match/20210-team-aster-vs-vici-gaming/')

In [61]:
predict = parse_live('https://dota2.ru/esport/match/20215-t1-vs-team-spirit/', 3)
pd.DataFrame(predict).to_csv(r'C:\Users\Максим\Dota 2_PY_Project\parse\predict.csv', index=False, header=True)

## Объединение файлов в один

In [8]:
excel_names = os.listdir(path="parse")

In [9]:
excel_names[0]

'Dota_2_matches_05_09_2021.csv'

In [10]:
pd.read_csv('parse/'+excel_names[0]).head(20)

,match,pinnacle,parimatch,ggbet,date,tournament,team left,players_left,team right,players_right,ban_left,ban_right,pick_left_team,pick_right_team,result_left_team,result_right_team,score
0,PuckChamp vs Fantastic Five,1.61-2.14,1.13-5.40,-,05 Сентября 18:10 2021,Dota 2 Champions League 2021 Season 3,PuckChamp,"['krylat', 'young G', 'meLes', 'Astral', 'Duka...",Fantastic Five,"['dream`', 'Askold', 'Shergarat', 'jAMES', 'Se...","['Beastmaster', 'Lycan', 'Io', 'Night Stalker'...","['Keeper of the Light', 'Pangolier', 'Invoker'...","['Gyrocopter', 'Lina', 'Bloodseeker', 'Shadow ...","['Morphling', 'Ember Spirit', 'Tidehunter', 'R...","<span class=""team__title-label"">Победа</span>",NaN,25 : 13
1,PuckChamp vs Fantastic Five,1.61-2.14,1.13-5.40,-,05 Сентября 18:10 2021,Dota 2 Champions League 2021 Season 3,PuckChamp,"['krylat', 'young G', 'meLes', 'Astral', 'Duka...",Fantastic Five,"['dream`', 'Askold', 'Blizzy', 'jAMES', 'Seleri']","['Night Stalker', 'Beastmaster', 'Slark', 'Sve...","['Keeper of the Light', 'Omniknight', 'Tidehun...","['Gyrocopter', 'Invoker', 'Sand King', 'Rubick...","['Bloodseeker', 'Batrider', 'Magnus', 'Windran...","<span class=""team__title-label"">Победа</span>",NaN,27 : 26
2,Spider Pigzs vs Team Empire,2.61-1.46,7.00-1.08,-,05 Сентября 15:10 2021,Dota 2 Champions League 2021 Season 3,Team Empire,"['Shachlo', 'Smiling Knight', 'Lorenof', 'sayu...",Spider Pigzs,"['lil pleb', 'No!ob', 'Funn1k', 'OmaR', 'LeBron']","['Keeper of the Light', 'Bane', 'Batrider', 'M...","['Lina', 'Omniknight', 'Shadow Fiend', 'Sven',...","['Vengeful Spirit', 'Medusa', 'Storm Spirit', ...","['Luna', 'Death Prophet', 'Bloodseeker', 'Lion...","<span class=""team__title-label"">Победа</span>",NaN,31 : 26
3,Spider Pigzs vs Team Empire,2.61-1.46,7.00-1.08,-,05 Сентября 15:10 2021,Dota 2 Champions League 2021 Season 3,Spider Pigzs,"['lil pleb', 'No!ob', 'Funn1k', 'OmaR', 'LeBron']",Team Empire,"['Shachlo', 'Smiling Knight', 'Lorenof', 'sayu...","['Vengeful Spirit', 'Omniknight', 'Winter Wyve...","['Keeper of the Light', 'Lina', 'Tidehunter', ...","['Sven', 'Snapfire', 'Beastmaster', 'Phoenix',...","['Bloodseeker', 'Sniper', 'Void Spirit', 'Lion...","<span class=""team__title-label"">Победа</span>",NaN,21 : 12
4,Spider Pigzs vs Team Empire,2.61-1.46,7.00-1.08,-,05 Сентября 15:10 2021,Dota 2 Champions League 2021 Season 3,Team Empire,"['Shachlo', 'Smiling Knight', 'Lorenof', 'sayu...",Spider Pigzs,"['lil pleb', 'No!ob', 'Funn1k', 'OmaR', 'LeBron']","['Keeper of the Light', 'Pangolier', 'Batrider...","['Lina', 'Shadow Demon', 'Beastmaster', 'Storm...","['Axe', 'Gyrocopter', 'Leshrac', 'Dark Willow'...","['Morphling', 'Void Spirit', 'Tidehunter', 'Ru...","<span class=""team__title-label"">Победа</span>",NaN,23 : 9
5,Team Aster vs Vici Gaming,1.48-2.68,1.02-11.00,-,05 Сентября 14:16 2021,i-League 2021 Season 2,Vici Gaming,"['The last dark', 'Ori', 'old eLeVeN', 'Pyw', ...",Team Aster,"['Monet', 'White丶Album_白学家', 'Xxs', 'Borax', '...","['Lina', 'Monkey King', 'Slark', 'Clinkz', 'Ur...","['Lion', 'Keeper of the Light', 'Bloodseeker',...","['Sven', 'Puck', 'Dragon Knight', 'Phoenix', '...","['Luna', 'Queen of Pain', 'Pangolier', 'Pudge'...","<span class=""team__title-label"">Победа</span>",NaN,21 : 15
6,Team Aster vs Vici Gaming,1.48-2.68,1.02-11.00,-,05 Сентября 14:16 2021,i-League 2021 Season 2,Team Aster,"['Monet', 'White丶Album_白学家', 'Xxs', 'Borax', '...",Vici Gaming,"['The last dark', 'Ori', 'old eLeVeN', 'Pyw', ...","['Lion', 'Keeper of the Light', 'Sven', 'Omnik...","['Monkey King', 'Lina', 'Juggernaut', 'Storm S...","['Medusa', 'Queen of Pain', 'Vengeful Spirit',...","['Anti-Mage', 'Void Spirit', 'Bloodseeker', 'P...","<span class=""team__title-label"">Победа</span>",NaN,39 : 26
7,Team Aster vs Vici Gaming,1.48-2.68,1.02-11.00,-,05 Сентября 14:16 2021,i-League 2021 Season 2,Vici Gaming,"['The last dark', 'Ori', 'old eLeVeN', 'Pyw', ...",Team Aster,"['Monet', 'White丶Album_白学家', 'Xxs', 'Borax', '...","['Monkey King', 'Lina', 'Storm Spirit', '

In [11]:
data_dir = Path("parse/")

In [12]:
df = pd.concat([pd.read_csv(f) for f in data_dir.glob("*.csv")], ignore_index=True)

In [13]:
df.to_csv(r'C:\Users\Максим\Dota 2_PY_Project\parse\data_final.csv', index=False, header=True)

## live

In [42]:
match = {'match': 'PuckChamp vs Team Unique',
         'pinnacle': '2.15-1.65',
         'parimatch': '2.03-1.73',
         'ggbet': '-',
         'date': '11 Сентября 18:00 2021',
         'tournament': 'Dota 2 Champions League 2021 Season 3',
         'team left': ['PuckChamp'],
         'players_left': [['krylat',
                          'young G',
                          'meLes',
                          'Astral',
                          'Dukalis']],
         'team right': ['Team Unique'],
         'players_right': [['spAce',
                           'fn',
                           'Petushara',
                           'Bignum',
                           'Slayer']],
         'ban_left': [['Night Stalker',
                      'Dawnbreaker',
                      'Gyrocopter',
                      'Medusa',
                      'Templar Assasin',
                      'Slark',
                      'Faceless Void']],
         'ban_right': [['Pangolier',
                       'Keeper Of The Light',
                       'Mars',
                       'Viper',
                       'Batrider',
                       'Rubick',
                       'Lion']],
         'pick_left_team': [['Sven',
                            'Lina',
                            'Enchantress',
                            'Tidehunter',
                            'Snapfire']],
         'pick_right_team': [['Shadow Demon',
                             'Beastmaster',
                             'Invoker',
                             'Bane',
                             'Monkey King']],
         'result_left_team': [None],
         'result_right_team': [None],
         'score': ['VS']}

In [43]:
pd.DataFrame(match).to_csv(r'C:\Users\Максим\Dota 2_PY_Project\parse\predict.csv', index=False, header=True)

## Parse live

In [44]:
match = parse_live('https://dota2.ru/esport/match/20279-v-gaming-vs-b8/', 1)

In [51]:
pd.DataFrame(match).to_csv(r'C:\Users\Максим\Dota 2_PY_Project\parse\predict.csv', index=False, header=True)

In [46]:
match

{'match': 'V-Gaming vs B8',
 'pinnacle': '1.18-4.53',
 'parimatch': '1.72-2.05',
 'ggbet': '-',
 'date': '11 Сентября 21:10 2021',
 'tournament': 'Dota 2 Champions League 2021 Season 3',
 'team left': ['V-Gaming'],
 'players_left': [['MALYW GRUT !', 'Naive-', 'destroyed', 'ArrOw', 'malady']],
 'team right': ['B8'],
 'players_right': [['watson', 'Dendi', 'Limitless', 'TSA', 'Gatciy']],
 'ban_left': [['Beastmaster',
   'Viper',
   'Clockwerk',
   'Vengeful Spirit',
   'Ember Spirit',
   'Tidehunter',
   'Legion Commander']],
 'ban_right': [['Disruptor',
   'Dawnbreaker',
   'Treant Protector',
   'Batrider',
   'Invoker',
   'Faceless Void',
   'Bloodseeker']],
 'pick_left_team': [['Shadow Fiend',
   'Weaver',
   'Night Stalker',
   'Lina',
   'Jakiro']],
 'pick_right_team': [['Monkey King',
   'Void Spirit',
   'Lycan',
   'Shadow Demon',
   'Elder Titan']],
 'result_left_team': [None],
 'result_right_team': [<span class="team__title-label">Победа</span>],
 'score': ['16 : 31']}